Explanation of the Code
Data Loading and Preparation:

load_dataset() function loads images and corresponding labels from the specified directory.
Images are resized to 256x256 pixels and normalized to the range [0, 1].
Data Augmentation:

Data augmentation is applied using ImageDataGenerator to improve the robustness of the model.
Model Building:

A Hybrid CNN-LSTM model is built using DenseNet-121 as the CNN backbone and an LSTM layer for capturing temporal dependencies.
The model outputs the hemorrhage volume as a regression task.
Model Training:

The model is compiled with the Adam optimizer and mean squared error loss function.
Training is performed with early stopping and model checkpointing to avoid overfitting.
Model Evaluation:

After training, the model is evaluated on the test set.
Mean Squared Error (MSE) and Mean Absolute Error (MAE) are calculated to assess the performance.
Make sure to replace 'path/to/images' and 'path/to/labels.csv' with the actual paths to your dataset. This code is designed to run in Google Colab, but it should work in any Python environment with the necessary libraries installed.

In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, LSTM, TimeDistributed, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score, confusion_matrix, classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from glob import glob

# Constants
IMAGE_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001

# Load the dataset
# Assuming dataset is downloaded and available in the current working directory
def load_dataset(image_dir, label_file):
    images = []
    labels = pd.read_csv(label_file)

    for idx, row in labels.iterrows():
        image_path = os.path.join(image_dir, row['filename'])
        if os.path.exists(image_path):
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
            images.append(img)
        else:
            print(f"Image {image_path} not found!")

    images = np.array(images).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
    labels = labels.drop(columns=['filename']).values

    return images, labels

# Preprocess the dataset
def preprocess_images(images):
    # Normalization
    images = images / 255.0
    return images

# Data augmentation
def augment_data(images, labels):
    datagen = ImageDataGenerator(rotation_range=20,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 horizontal_flip=True)
    datagen.fit(images)
    return datagen

# Load and preprocess dataset
image_dir = 'path/to/images'
label_file = 'path/to/labels.csv'
images, labels = load_dataset(image_dir, label_file)
images = preprocess_images(images)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Augment the training data
datagen = augment_data(X_train, y_train)

# Build the Hybrid CNN-LSTM Model
def build_model(input_shape):
    input_layer = Input(shape=input_shape)

    # CNN Backbone (DenseNet121)
    base_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=input_layer)
    x = base_model.output
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    cnn_output = Dense(256, activation='relu')(x)

    # Reshape for LSTM
    lstm_input = tf.reshape(cnn_output, [-1, 1, 256])

    # LSTM Layer
    lstm_output = LSTM(128, return_sequences=False)(lstm_input)

    # Output Layer (Regression for volume quantification)
    output_layer = Dense(1, activation='linear')(lstm_output)

    model = Model(inputs=input_layer, outputs=output_layer)

    return model

model = build_model((IMAGE_SIZE, IMAGE_SIZE, 1))
model.summary()

# Compile the model
optimizer = Adam(lr=LEARNING_RATE)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')

# Train the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=len(X_train) // BATCH_SIZE,
                    validation_data=(X_val, y_val),
                    epochs=EPOCHS,
                    callbacks=[early_stopping, model_checkpoint])

# Evaluate the model
model.load_weights('best_model.h5')
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mse = np.mean((y_test - y_pred)**2)
mae = np.mean(np.abs(y_test - y_pred))

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")

# Visualization
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate classification performance
# Assuming classification labels are available in the dataset
# Convert regression outputs to binary classification
y_pred_class = np.where(y_pred > 0.5, 1, 0)
accuracy = accuracy_score(y_test, y_pred_class)
precision = precision_score(y_test, y_pred_class)
recall = recall_score(y_test, y_pred_class)
roc_auc = roc_auc_score(y_test, y_pred)
average_precision = average_precision_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"ROC-AUC: {roc_auc}")
print(f"Average Precision: {average_precision}")

# Confusion matrix and classification report
cm = confusion_matrix(y_test, y_pred_class)
cr = classification_report(y_test, y_pred_class)

print("Confusion Matrix:")
print(cm)
print("Classification Report:")
print(cr)

# Plot ROC curve
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Plot Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_test, y_pred)
plt.figure()
plt.plot(recall, precision, color='blue', lw=2, label='PR curve (area = %0.2f)' % average_precision)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/labels.csv'